In [1]:
import pandas as pd

In [ ]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'casper',
    'database': 'bolero'
}

IN_FILE = '../data/dump.json'

composer_columns = ['composer_id', 'name', 'complete_name', 'birth', 'death', 'epoch', 'recommended', 'popular']
composers = pd.DataFrame(columns=composer_columns)

works_columns = ['work_id', 'composer_id', 'title', 'subtitle', 'searchterms', 'genre', 'recommended', 'popular']
works = pd.DataFrame(columns=works_columns)

def read():
    global composers
    global works
    raw_composers = pd.read_json(IN_FILE, orient='records')
    for index, raw_composer in raw_composers.iterrows():
        print('Processing', raw_composer['name'], index)
        composer = {'composer_id': index, **raw_composer}
        composers = pd.concat([composers, pd.DataFrame([composer]).filter(composer_columns)], ignore_index=True)
        raw_works = composer['works']
        cnt = 0
        for raw_work in raw_works:
            work = {'work_id': cnt, 'composer_id': index, **raw_work}
            cnt += 1
            works = pd.concat([works, pd.DataFrame([work])], ignore_index=True)
    composers = composers.where(pd.notnull(composers), None)
    works = works.where(pd.notnull(works), None)

read()

### Composer

In [7]:
print('recommended cnt:', (composers['recommended'] == 1).sum())
print('popular cnt:', (composers['popular'] == 1).sum())
print()
print('epoch:', composers['epoch'].value_counts())

recommended cnt: 77
popular cnt: 23

epoch: epoch
20th Century      49
Late Romantic     39
Romantic          36
Post-War          27
Baroque           22
Renaissance       17
Early Romantic    13
Classical          9
21st Century       4
Medieval           4
Name: count, dtype: int64


### Work

In [13]:
print('recommended:',works['recommended'].value_counts())
print()
print('popular:', works['popular'].value_counts())
print()
print('genre:', works['genre'].value_counts())

recommended: recommended
0    23741
1     1234
Name: count, dtype: int64

popular: popular
0    24578
1      397
Name: count, dtype: int64

genre: genre
Vocal         8509
Keyboard      5266
Orchestral    5120
Chamber       4806
Stage         1274
Name: count, dtype: int64


In [16]:
import requests

def get_composer_nationality(composer_name, language="en"):
    search_url = "https://www.wikidata.org/w/api.php"
    search_params = {
        "action": "wbsearchentities",
        "format": "json",
        "language": language,    # 搜索所用语言
        "search": composer_name
    }

    try:
        response = requests.get(search_url, params=search_params, timeout=10)
        data = response.json()

        # 若检索成功且有结果，取首条
        if "search" in data and len(data["search"]) > 0:
            qid = data["search"][0]["id"]  # 例如 "Q254"
        else:
            print(f"[未找到匹配项] {composer_name}")
            return None
    except Exception as e:
        print(f"[搜索异常] {composer_name} - {e}")
        return None

    # Step 2: 根据 Q ID 获取实体信息（特别是 claims 中的 P27（国籍））
    entity_url = "https://www.wikidata.org/w/api.php"
    entity_params = {
        "action": "wbgetentities",
        "format": "json",
        "ids": qid,
        "props": "claims|labels",  # 同时拉取 claims 和 labels
        "languages": language
    }

    try:
        response = requests.get(entity_url, params=entity_params, timeout=10)
        data = response.json()

        # 解析国籍属性 P27
        entity_data = data["entities"].get(qid, {})
        claims = entity_data.get("claims", {})

        if "P27" not in claims:
            print(f"[无国籍信息] {composer_name} (QID={qid})")
            return None

        # 可能存在多个国籍，这里只演示取第一个
        country_claim = claims["P27"][0]
        country_datavalue = (country_claim["mainsnak"]
                             .get("datavalue", {})
                             .get("value", {}))

        country_qid = country_datavalue.get("id")  # 国籍对应的 Q ID（如 "Q40"=Austria）
        if not country_qid:
            print(f"[国籍数据异常] {composer_name} (QID={qid})")
            return None

        # 为了获取“国籍”的文字，需要进一步从 labels 中取
        # 如果之前拉取了 labels，检查是否已经包含 country_qid 的信息
        # 但通常 labels 中只包含当前实体的 label，没有包含 country 的 label。
        # 因此需要再次查询 country_qid。

        country_entity_params = {
            "action": "wbgetentities",
            "format": "json",
            "ids": country_qid,
            "props": "labels",
            "languages": language
        }
        resp_country = requests.get(entity_url, params=country_entity_params, timeout=10).json()

        country_entity = resp_country["entities"].get(country_qid, {})
        labels = country_entity.get("labels", {})

        # 从 label 中获取文本，例如 "Austria"
        if language in labels and "value" in labels[language]:
            country_label = labels[language]["value"]
        else:
            # 如果该语言下无 label，可尝试 "en" 或其他回退
            country_label = labels.get("en", {}).get("value", None)

        return country_label

    except Exception as e:
        print(f"[实体查询异常] {composer_name} (QID={qid}) - {e}")
        return None


# =============== 测试示例 ==================
if __name__ == "__main__":
    test_composers = [
        "Wolfgang Amadeus Mozart",    # 奥地利作曲家
        "Pyotr Ilyich Tchaikovsky",  # 俄罗斯作曲家
        "Johann Sebastian Bach",     # 德国作曲家
        "Frederic Chopin",           # 波兰/法国（多国籍）
        "Giacomo Puccini",           # 意大利作曲家
        "xxxx yyy"                   # 测试一个不存在或不知名的名字
    ]

    for composer in test_composers:
        nationality = get_composer_nationality(composer)
        print(f"{composer} -> {nationality}")

Wolfgang Amadeus Mozart -> Archduchy of Austria
Pyotr Ilyich Tchaikovsky -> Russian Empire
Johann Sebastian Bach -> Saxe-Eisenach
Frederic Chopin -> France
Giacomo Puccini -> Kingdom of Italy
[未找到匹配项] xxxx yyy
xxxx yyy -> None
